### HW9

Создать веб-сервис на Flask через Google Colab или на локальной машине. Научиться отправлять post-запросы с помощью Postman и получать на них ответы. Можно использовать любую из обученных моделей в заданиях ко 2 и 3 занятиям.

## Схема проекта

**Принципиальные моменты:**
*   У страховой компании нет времени "на подумать", решение о приеме на страхование или отказе, а также о тарифе должно быть сделано непосредственно в момент обращения клиента за полисом. Соответственно, сервис должен работать в режиме реального времени и давать ответ в течение максимум нескольких секунд.
*   Сервис скоринга строится обычно отдельным блоком, а фронт-система взаимодействует с сервисом через API (Application programming interface).
*   Под каждый блок, как правило, - свое программное решение.
*   Запрос данных из внешних источников может производиться как на уровне фронт-системы, так и на уровне сервиса.
*   Один из возможных вариантов: код на Python + Flask + WSGI (взаимодействие между программой на python и веб-сервером) + Web Server. Передача информации производится путем post-запросов и обмена файлами json.
*   Для тестирования можно использовать Postman.
*   Также бывают варианты:
    *   Внедрение моделей в сам код в виде функций;
    *   docker + grpc
    *   Sparc ML lib + сериализаторы и десериализаторы
    *   PMML
    *   Flask + Celery

  ![](https://drive.google.com/uc?export=view&id=1OAOF1M2U14UJWDmeJg2mwo-pgSwyNyzc)

Исследование:<br/>
  ![](https://drive.google.com/uc?export=view&id=1dnMajOh1EWbjKFIDfvmqYcux1yTdKEJO)
<br/>
Разработка:<br/>
  ![](https://drive.google.com/uc?export=view&id=1DXa3EhgIQXPzfINAETrEhi7lt3NjLR4o)
<br/>
Тестирование:<br/>
  ![](https://drive.google.com/uc?export=view&id=1wQsJ0pipWsjDLfzGOX3n4KMLpn21PNgh)

*   Предобработчики данных - это часть модели ML и должны лежать рядом с моделью, в том числе для синхронизации версий.
*   Препроцессинг внешних данных может быть вынесен в отдельные блоки, поддерживающие интеграцию со сторонними api.
*   Вновь поступающие данные можно доразмечать и отправлять на дообучение модели.
*   Доступ к данным в компании должен быть простым и идентичным у различных пользователей (отделов, департаментов). Лучшая практика - создание DWH. 

*   Модель ML - это программный код.
*   Может быть написан на разных языках программирования.
*   Сейчас чаще код пишут на Python, хотя "под капотом" библиотек может быть и Java, и C.
*   Для передачи между участниками процесса может быть выполнена сериализация модели (перевод в последовательность битов - json или xml), используются библиотеки pickle, drill

**Общие концепции сервинга:**
*   Ввести идентификатор модели
*   В папке хранить сериализованную модель и веса коэффициентов
*   Загружать модель в память при старте сервиса
*   Обрабатывать входящие запросы через http и вызывать модель
*   Для картинок можно использовать grpc
*   Фичи, как правило, передаются в теле запроса post
*   На каждую модель не нужно писать отдельный сервис, а подгружать модель из папки
*   При сохранении модель обязательно сопровождать документацией

## JSON

JSON (JavaScript Object Notation) - простой текстовый формат обмена данными, он основан на подмножестве языка программирования JavaScript.<br/>
Например, строка из нашего датасета выглядела бы следующим образом:<br/>
```
{
"ID": 1,
"Exposure": 0.583,
"RecordBeg": "2004-06-01",
"RecordEnd": "",
"DrivAge": 55,
"Gender": "Female",
...
}
```

## При внедрении

**При внедрении необходимо сделать:**
*   Определить формат json'а, в котором данные будут приниматься сервисом и отправляться обратно.
*   Определить ip-адрес и порт, на который будут поступать данные.
*   Создать во Flask необходимые роуты:<br/>
    `@app.route('/predict_example', method='POST')`<br/>
    `def predict_example():`
*   Перенести во Flask все функции преобразования данных,
    *   формат данных, приходящих от фронт-системы, может отличаться от формата исторических данных, использовавшихся при построении модели; в результате преобразований данные на вход модели должны поступить ровно в том виде, в каком была обучена модель.
*   Загрузить обученные модели.
*   Настроить логирование, запись котировок.

**Особенности:**
*   Библиотека H2O использует виртуальную Java-машину:
    *   ее нужно инициализировать один раз, а не поднимать заново для каждого расчета;
    *   вручную выделить под нее отдельный порт и указать размер используемой памяти;
    *   не создавать каждый раз заново H2O-Frame, а записывать в единожды подготовленный.
*   Необходимо удостовериться, что на всех этапах сервис отрабатывает корректно; например, можно иметь заготовленный массив котировок с заранее известными ответами.
*   Необходимо провести нагрузочное тестирование и удостовериться, что сервис справляется с нагрузкой.
*   Для согласованности версий Python, Java при переносе на другие серверы имеет смысл использовать докеры.



## Flask

Google Colab provides a virtual machine so we cannot access the localhost as we do on our local machine when running a local web server. What we can do is expose it to a public URL using ngrok.
https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df = pd.read_csv('/content/drive/My Drive/ds_in_prodact/freMPL-R.csv', low_memory=False)

In [6]:
!pip install flask-ngrok

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [8]:
# Пробный запуск Flask

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9ace4b00.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Feb/2020 15:56:52] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [15/Feb/2020 15:56:52] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Feb/2020 15:56:55] "GET /a HTTP/1.1" 200 -


В качестве примера используем обученную обобщенную линейную модель (GLM) и библиотеку H2O.

Преобразование данных и обучение модели были проведены на втором занятии. Сейчас воспользуемся готовым результатом.

In [12]:
!pip install h2o

     |████████████████████████████████| 126.2MB 77kB/s 
  Created wheel for h2o: filename=h2o-3.28.0.3-py2.py3-none-any.whl size=126303006 sha256=0f47c1a66247156601048b4947c64dc2f04618ccea40f653ef4e333239edcdcd
  Stored in directory: /root/.cache/pip/wheels/fa/7c/ce/95ae52b4d3f1b14a27c3c961c1f94635aee841ab1eec3aeeca
Successfully built h2o


In [13]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpobcobvsb
  JVM stdout: /tmp/tmpobcobvsb/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpobcobvsb/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_unknownUser_ko75k5
H2O cluster total nodes:,1
H2O cluster free memory:,3 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [0]:
# Загружаем обученные модели

model_glm_binomial = h2o.load_model('/content/drive/My Drive/ds_in_prodact/GLM_model_python_1581783997770_1')

In [18]:
# Запуск Flask

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route('/predict', methods=['GET', 'POST'])
def predict3():

    try:

      json_input = request.json

      ID = json_input["ID"]
      LicAge = json_input["LicAge"]
      Gender = map_for_dict_Gender(json_input["Gender"])
      MariStat = map_for_dict_MariStat(json_input["MariStat"])
      DrivAge = json_input["DrivAge"]
      HasKmLimit = json_input["HasKmLimit"]
      BonusMalus = json_input["BonusMalus"]
      OutUseNb = json_input["OutUseNb"]
      RiskArea = json_input["RiskArea"]
      VehUsg_Private = f_VehUsage_Private(json_input["VehUsage"])
      VehUsg_Private_trip_to_office = f_VehUsage_Private_trip_to_office(json_input["VehUsage"])
      VehUsg_Professional = f_VehUsage_Professional(json_input["VehUsage"])
      VehUsg_Professional_run = f_VehUsage_Professional_run(json_input["VehUsage"])
      CSP1 = 0
      CSP2 = 0
      CSP3 = 0
      CSP6 = 0
      CSP7 = 0
      CSP20 = 0
      CSP21 = 0
      CSP22 = 0
      CSP26 = 0
      CSP37 = 0
      CSP40 = 0
      CSP42 = 0
      CSP46 = 0
      CSP47 = 0
      CSP48 = 0
      CSP49 = 0
      CSP50 = 0
      CSP55 = 0
      CSP56 = 0
      CSP57 = 0
      CSP60 = 0
      CSP65 = 0
      CSP66 = 0

      hf = return_NewH2o_Frame()

      hf[0, 'LicAge'] = LicAge
      hf[0, 'Gender'] = Gender
      hf[0, 'MariStat'] = MariStat
      hf[0, 'DrivAge'] = DrivAge
      hf[0, 'HasKmLimit'] = HasKmLimit
      hf[0, 'BonusMalus'] = BonusMalus
      hf[0, 'OutUseNb'] = OutUseNb
      hf[0, 'RiskArea'] = RiskArea
      hf[0, 'VehUsg_Private'] = VehUsg_Private
      hf[0, 'VehUsg_Private+trip to office'] = VehUsg_Private_trip_to_office
      hf[0, 'VehUsg_Professional'] = VehUsg_Professional
      hf[0, 'VehUsg_Professional run'] = VehUsg_Professional_run
      hf[0, 'CSP1'] = CSP1
      hf[0, 'CSP2'] = CSP2
      hf[0, 'CSP3'] = CSP3
      hf[0, 'CSP6'] = CSP6
      hf[0, 'CSP7'] = CSP7
      hf[0, 'CSP20'] = CSP20
      hf[0, 'CSP21'] = CSP21
      hf[0, 'CSP22'] = CSP22
      hf[0, 'CSP26'] = CSP26
      hf[0, 'CSP37'] = CSP37
      hf[0, 'CSP40'] = CSP40
      hf[0, 'CSP42'] = CSP42
      hf[0, 'CSP46'] = CSP46
      hf[0, 'CSP47'] = CSP47
      hf[0, 'CSP48'] = CSP48
      hf[0, 'CSP49'] = CSP49
      hf[0, 'CSP50'] = CSP50
      hf[0, 'CSP55'] = CSP55
      hf[0, 'CSP56'] = CSP56
      hf[0, 'CSP57'] = CSP57
      hf[0, 'CSP60'] = CSP60
      hf[0, 'CSP65'] = CSP65
      hf[0, 'CSP66'] = CSP66

      prediction_Binomial = model_glm_binomial.predict(hf)
      value_Binomial  = prediction_Binomial.as_data_frame()['predict'][0]

      return jsonify({'ID':ID, 'value_Binomial':value_Binomial}) 
    
    except:
      
      return "Error"



if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://53779676.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Feb/2020 16:41:21] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2020 16:41:21] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2020 16:41:22] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2020 16:41:22] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2020 16:41:31] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2020 16:41:31] "POST /predict HTTP/1.1" 200 -


##Postman

Для тестирования сервиса через API удобно использовать программу Postman https://www.postman.com/

Создаем тестовый json со значениями параметров в соответствии с обученной моделью<br/>
```
{
"ID":1,
"LicAge":468,
"RecordBeg":"2004-01-01",
"RecordEnd":"",
"VehAge":"",
"Gender":"Male",
"MariStat":"Other",
"SocioCateg":"CSP50",
"VehUsage":"Private",
"DrivAge":67,
"HasKmLimit":0,
"BonusMalus":50,
"OutUseNb":0,
"RiskArea":0
}
```

Запускаем Postman, выбираем VALUE = 'application/json'

  ![](https://drive.google.com/uc?export=view&id=16z29a8xAmRrRXaEb5mpjufi458W5WAsZ)

Формируем тестовый post-запрос

  ![](https://drive.google.com/uc?export=view&id=1enCft4k135BYAH01UgAsSX8Tj7LxaS0b)

Отправляем post-запрос на url Flask (см. выше)

  ![](https://drive.google.com/uc?export=view&id=143XO4e9NfH46APwPLwILDFcNmM3wtACj)

Получаем ответ от сервиса

  ![](https://drive.google.com/uc?export=view&id=1ZjRp5Pys_Bg2m1KM4iWtczCdIxx_zCIv)

При обучении новых моделей (даже на той же тренировочной выборке) коэффициенты и, соответственно, прогнозные значения могут отличаться от приведенных выше

##Дополнительно

*   Логирование
    *   Пришел запрос
    *   Что пришло
    *   Ушел ответ
    *   Что ушло
    *   Средства зависят от системы логирования, ELK stack (elasticsearch, logstash, kibana)
*   requirements.txt
*   pip freeze
   ![](https://drive.google.com/uc?export=view&id=1b2LKW_kYSTUPUcLV3i3KqWmGqzv_hGh5)
*   Многопоточность
    *   Изначально веб-серверы на Python работают в developer mode
    *   Из-за GIL (python global interpreter lock) активен только один поток (в один момент времени может активно обрабатываться только один запрос)
    *   Flask, Django production mode (WSGI - программный интерфейс для питоновских программ)
    *   Веб-сервер сам параллелит запросы
*   Сокращение времени вывода модели в прод:
    *   Data Scientist должен предоставить скрипт, который легко встроить в веб-вервис
    *   Сервинг - процесс перевода модели в продукцию, в рамках которого происходит сохранение функций и обработчиков в файл
*   Контейнеризация:
    *   Kubernetes + docker
    *   Масштабирование приложений, не имеющих изменяемого состояния
    *   Запросы распределяются на различные приложения
*   Серьезный прод - grpc:
    *   grpc в 2 слоя
    *   первый слой: grpc + модели, поднимаются контейнеры с моделями, которые могут принимать запросы на инференс
    *   второй слой: поднимается прокси на чем-нибудь асинхронном (grpc-gateway, go, aiohttp), у которого снаружи http, а сам он делает грс вызовы по grpc
    *   плюсы такой схемы: можно независимо скейлить модели и gateway, делатб AB-тесты через canary deployment и sticky routing для пользователя
    *   grpc позволяет делать bidirectional streaming, когда можно принять за определенное временное окно пачку запросов, сложить их в батч и сделать по ним инференс, а потом поштучно отстсрелить
    *   если модель делает запросы к внешним источникам, то хороший вариант - aiohttp, а инференс можно сделать через 'run_in_executor' в пул процессов, если сериализация контекста не является затратной 

In [ ]:
## Схема проекта

**Принципиальные моменты:**
*   У страховой компании нет времени "на подумать", решение о приеме на страхование или отказе, а также о тарифе должно быть сделано непосредственно в момент обращения клиента за полисом. Соответственно, сервис должен работать в режиме реального времени и давать ответ в течение максимум нескольких секунд.
*   Сервис скоринга строится обычно отдельным блоком, а фронт-система взаимодействует с сервисом через API (Application programming interface).
*   Под каждый блок, как правило, - свое программное решение.
*   Запрос данных из внешних источников может производиться как на уровне фронт-системы, так и на уровне сервиса.
*   Один из возможных вариантов: код на Python + Flask + WSGI (взаимодействие между программой на python и веб-сервером) + Web Server. Передача информации производится путем post-запросов и обмена файлами json.
*   Для тестирования можно использовать Postman.
*   Также бывают варианты:
    *   Внедрение моделей в сам код в виде функций;
    *   docker + grpc
    *   Sparc ML lib + сериализаторы и десериализаторы
    *   PMML
    *   Flask + Celery

  ![](https://drive.google.com/uc?export=view&id=1OAOF1M2U14UJWDmeJg2mwo-pgSwyNyzc)

Исследование:<br/>
  ![](https://drive.google.com/uc?export=view&id=1dnMajOh1EWbjKFIDfvmqYcux1yTdKEJO)
<br/>
Разработка:<br/>
  ![](https://drive.google.com/uc?export=view&id=1DXa3EhgIQXPzfINAETrEhi7lt3NjLR4o)
<br/>
Тестирование:<br/>
  ![](https://drive.google.com/uc?export=view&id=1wQsJ0pipWsjDLfzGOX3n4KMLpn21PNgh)

*   Предобработчики данных - это часть модели ML и должны лежать рядом с моделью, в том числе для синхронизации версий.
*   Препроцессинг внешних данных может быть вынесен в отдельные блоки, поддерживающие интеграцию со сторонними api.
*   Вновь поступающие данные можно доразмечать и отправлять на дообучение модели.
*   Доступ к данным в компании должен быть простым и идентичным у различных пользователей (отделов, департаментов). Лучшая практика - создание DWH. 

*   Модель ML - это программный код.
*   Может быть написан на разных языках программирования.
*   Сейчас чаще код пишут на Python, хотя "под капотом" библиотек может быть и Java, и C.
*   Для передачи между участниками процесса может быть выполнена сериализация модели (перевод в последовательность битов - json или xml), используются библиотеки pickle, drill

**Общие концепции сервинга:**
*   Ввести идентификатор модели
*   В папке хранить сериализованную модель и веса коэффициентов
*   Загружать модель в память при старте сервиса
*   Обрабатывать входящие запросы через http и вызывать модель
*   Для картинок можно использовать grpc
*   Фичи, как правило, передаются в теле запроса post
*   На каждую модель не нужно писать отдельный сервис, а подгружать модель из папки
*   При сохранении модель обязательно сопровождать документацией

## JSON

JSON (JavaScript Object Notation) - простой текстовый формат обмена данными, он основан на подмножестве языка программирования JavaScript.<br/>
Например, строка из нашего датасета выглядела бы следующим образом:<br/>
```
{
"ID": 1,
"Exposure": 0.583,
"RecordBeg": "2004-06-01",
"RecordEnd": "",
"DrivAge": 55,
"Gender": "Female",
...
}
```

## При внедрении

**При внедрении необходимо сделать:**
*   Определить формат json'а, в котором данные будут приниматься сервисом и отправляться обратно.
*   Определить ip-адрес и порт, на который будут поступать данные.
*   Создать во Flask необходимые роуты:<br/>
    `@app.route('/predict_example', method='POST')`<br/>
    `def predict_example():`
*   Перенести во Flask все функции преобразования данных,
    *   формат данных, приходящих от фронт-системы, может отличаться от формата исторических данных, использовавшихся при построении модели; в результате преобразований данные на вход модели должны поступить ровно в том виде, в каком была обучена модель.
*   Загрузить обученные модели.
*   Настроить логирование, запись котировок.

**Особенности:**
*   Библиотека H2O использует виртуальную Java-машину:
    *   ее нужно инициализировать один раз, а не поднимать заново для каждого расчета;
    *   вручную выделить под нее отдельный порт и указать размер используемой памяти;
    *   не создавать каждый раз заново H2O-Frame, а записывать в единожды подготовленный.
*   Необходимо удостовериться, что на всех этапах сервис отрабатывает корректно; например, можно иметь заготовленный массив котировок с заранее известными ответами.
*   Необходимо провести нагрузочное тестирование и удостовериться, что сервис справляется с нагрузкой.
*   Для согласованности версий Python, Java при переносе на другие серверы имеет смысл использовать докеры.



## Flask

Google Colab provides a virtual machine so we cannot access the localhost as we do on our local machine when running a local web server. What we can do is expose it to a public URL using ngrok.
https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/ds_in_prodact/freMPL-R.csv', low_memory=False)

!pip install flask-ngrok

from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

# Пробный запуск Flask

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run()

В качестве примера используем обученную обобщенную линейную модель (GLM) и библиотеку H2O.

Преобразование данных и обучение модели были проведены на втором занятии. Сейчас воспользуемся готовым результатом.

!pip install h2o

import h2o
h2o.init()

# Загружаем обученные модели

model_glm_binomial = h2o.load_model('/content/drive/My Drive/ds_in_prodact/GLM_model_python_1581783997770_1')

# Запуск Flask

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route('/predict', methods=['GET', 'POST'])
def predict3():

    try:

      json_input = request.json

      ID = json_input["ID"]
      LicAge = json_input["LicAge"]
      Gender = map_for_dict_Gender(json_input["Gender"])
      MariStat = map_for_dict_MariStat(json_input["MariStat"])
      DrivAge = json_input["DrivAge"]
      HasKmLimit = json_input["HasKmLimit"]
      BonusMalus = json_input["BonusMalus"]
      OutUseNb = json_input["OutUseNb"]
      RiskArea = json_input["RiskArea"]
      VehUsg_Private = f_VehUsage_Private(json_input["VehUsage"])
      VehUsg_Private_trip_to_office = f_VehUsage_Private_trip_to_office(json_input["VehUsage"])
      VehUsg_Professional = f_VehUsage_Professional(json_input["VehUsage"])
      VehUsg_Professional_run = f_VehUsage_Professional_run(json_input["VehUsage"])
      CSP1 = 0
      CSP2 = 0
      CSP3 = 0
      CSP6 = 0
      CSP7 = 0
      CSP20 = 0
      CSP21 = 0
      CSP22 = 0
      CSP26 = 0
      CSP37 = 0
      CSP40 = 0
      CSP42 = 0
      CSP46 = 0
      CSP47 = 0
      CSP48 = 0
      CSP49 = 0
      CSP50 = 0
      CSP55 = 0
      CSP56 = 0
      CSP57 = 0
      CSP60 = 0
      CSP65 = 0
      CSP66 = 0

      hf = return_NewH2o_Frame()

      hf[0, 'LicAge'] = LicAge
      hf[0, 'Gender'] = Gender
      hf[0, 'MariStat'] = MariStat
      hf[0, 'DrivAge'] = DrivAge
      hf[0, 'HasKmLimit'] = HasKmLimit
      hf[0, 'BonusMalus'] = BonusMalus
      hf[0, 'OutUseNb'] = OutUseNb
      hf[0, 'RiskArea'] = RiskArea
      hf[0, 'VehUsg_Private'] = VehUsg_Private
      hf[0, 'VehUsg_Private+trip to office'] = VehUsg_Private_trip_to_office
      hf[0, 'VehUsg_Professional'] = VehUsg_Professional
      hf[0, 'VehUsg_Professional run'] = VehUsg_Professional_run
      hf[0, 'CSP1'] = CSP1
      hf[0, 'CSP2'] = CSP2
      hf[0, 'CSP3'] = CSP3
      hf[0, 'CSP6'] = CSP6
      hf[0, 'CSP7'] = CSP7
      hf[0, 'CSP20'] = CSP20
      hf[0, 'CSP21'] = CSP21
      hf[0, 'CSP22'] = CSP22
      hf[0, 'CSP26'] = CSP26
      hf[0, 'CSP37'] = CSP37
      hf[0, 'CSP40'] = CSP40
      hf[0, 'CSP42'] = CSP42
      hf[0, 'CSP46'] = CSP46
      hf[0, 'CSP47'] = CSP47
      hf[0, 'CSP48'] = CSP48
      hf[0, 'CSP49'] = CSP49
      hf[0, 'CSP50'] = CSP50
      hf[0, 'CSP55'] = CSP55
      hf[0, 'CSP56'] = CSP56
      hf[0, 'CSP57'] = CSP57
      hf[0, 'CSP60'] = CSP60
      hf[0, 'CSP65'] = CSP65
      hf[0, 'CSP66'] = CSP66

      prediction_Binomial = model_glm_binomial.predict(hf)
      value_Binomial  = prediction_Binomial.as_data_frame()['predict'][0]

      return jsonify({'ID':ID, 'value_Binomial':value_Binomial}) 
    
    except:
      
      return "Error"



if __name__ == '__main__':
    app.run()

##Postman

Для тестирования сервиса через API удобно использовать программу Postman https://www.postman.com/

Создаем тестовый json со значениями параметров в соответствии с обученной моделью<br/>
```
{
"ID":1,
"LicAge":468,
"RecordBeg":"2004-01-01",
"RecordEnd":"",
"VehAge":"",
"Gender":"Male",
"MariStat":"Other",
"SocioCateg":"CSP50",
"VehUsage":"Private",
"DrivAge":67,
"HasKmLimit":0,
"BonusMalus":50,
"OutUseNb":0,
"RiskArea":0
}
```

Запускаем Postman, выбираем VALUE = 'application/json'

  ![](https://drive.google.com/uc?export=view&id=16z29a8xAmRrRXaEb5mpjufi458W5WAsZ)

Формируем тестовый post-запрос

  ![](https://drive.google.com/uc?export=view&id=1enCft4k135BYAH01UgAsSX8Tj7LxaS0b)

Отправляем post-запрос на url Flask (см. выше)

  ![](https://drive.google.com/uc?export=view&id=143XO4e9NfH46APwPLwILDFcNmM3wtACj)

Получаем ответ от сервиса

  ![](https://drive.google.com/uc?export=view&id=1ZjRp5Pys_Bg2m1KM4iWtczCdIxx_zCIv)

При обучении новых моделей (даже на той же тренировочной выборке) коэффициенты и, соответственно, прогнозные значения могут отличаться от приведенных выше

##Дополнительно

*   Логирование
    *   Пришел запрос
    *   Что пришло
    *   Ушел ответ
    *   Что ушло
    *   Средства зависят от системы логирования, ELK stack (elasticsearch, logstash, kibana)
*   requirements.txt
*   pip freeze
   ![](https://drive.google.com/uc?export=view&id=1b2LKW_kYSTUPUcLV3i3KqWmGqzv_hGh5)
*   Многопоточность
    *   Изначально веб-серверы на Python работают в developer mode
    *   Из-за GIL (python global interpreter lock) активен только один поток (в один момент времени может активно обрабатываться только один запрос)
    *   Flask, Django production mode (WSGI - программный интерфейс для питоновских программ)
    *   Веб-сервер сам параллелит запросы
*   Сокращение времени вывода модели в прод:
    *   Data Scientist должен предоставить скрипт, который легко встроить в веб-вервис
    *   Сервинг - процесс перевода модели в продукцию, в рамках которого происходит сохранение функций и обработчиков в файл
*   Контейнеризация:
    *   Kubernetes + docker
    *   Масштабирование приложений, не имеющих изменяемого состояния
    *   Запросы распределяются на различные приложения
*   Серьезный прод - grpc:
    *   grpc в 2 слоя
    *   первый слой: grpc + модели, поднимаются контейнеры с моделями, которые могут принимать запросы на инференс
    *   второй слой: поднимается прокси на чем-нибудь асинхронном (grpc-gateway, go, aiohttp), у которого снаружи http, а сам он делает грс вызовы по grpc
    *   плюсы такой схемы: можно независимо скейлить модели и gateway, делатб AB-тесты через canary deployment и sticky routing для пользователя
    *   grpc позволяет делать bidirectional streaming, когда можно принять за определенное временное окно пачку запросов, сложить их в батч и сделать по ним инференс, а потом поштучно отстсрелить
    *   если модель делает запросы к внешним источникам, то хороший вариант - aiohttp, а инференс можно сделать через 'run_in_executor' в пул процессов, если сериализация контекста не является затратной 